In [ ]:
# Input information
# NOTE: the file name is just a file the author used for testing, the file is not included in the repository

# SA2-139E Ph1
meeting = {
    'file':  "2020.06.08 203046 email summary.csv",
    'title': "SA2-139E Ph1",
    'dates': ['2020-05-31 12:00','2020-06-12 19:00']
}

# SA2-138E
#meeting = {
#    'file':  "2020.04.24 153334 email summary.csv",
#    'title': "SA2-138E",
#    'dates': ['2020-04-19 12:00','2020-04-24 19:00']
#}

# SA2-137E
#meeting = {
#    'file':  "2020.03.02 080314 email summary.csv",
#    'title': "SA2-137E",
#    'dates': ['2020-02-23 12:00','2020-02-28 19:00']
#}

In [ ]:
# You will need pandas/numpy to run the first part of the script. Then plotly for the second part. You can install plotly via pip (pip install plotly)
# If the PNG export is not working, most probably orca is not installed. You can install it by executing in the Anaconda cmd: conda install -c plotly plotly-orca
import pandas as pd
import numpy as np
import plotly.express as px

In [ ]:
# Create variables
email_dump    = meeting["file"]
title         = meeting["title"]
meeting_dates = meeting["dates"]

# Create DataFrame for analysis
df = pd.read_csv(
    email_dump,
    index_col=0)
df['date']    = pd.to_datetime(df['date'])
df['day']     = df['date'].dt.floor('d')
df['hour']    = df['date'].dt.floor('h')
df['company'] = df['sender address'].str.split('@', n = -1, expand = False).apply(lambda x: x[-1])

In [ ]:
df

In [ ]:
email_count_per_ai   = df.groupby('ai').count()
email_count_per_day  = df.groupby('day').count()
email_count_per_hour = df.groupby('hour').count()

In [ ]:
max_rows = pd.options.display.max_rows
pd.options.display.max_rows = None
emails_count = df.groupby('sender name').count().sort_values(by='date', ascending=False)
# See https://stackoverflow.com/questions/49574817/fill-a-new-pandas-column-with-row-numbers
emails_count['ranking'] = np.arange(len(emails_count)) + 1
emails_count['emails'] = emails_count['date']
display(emails_count[['ranking', 'emails']].iloc[0:25,:])
pd.options.display.max_rows = 60

In [ ]:
max_rows = pd.options.display.max_rows
pd.options.display.max_rows = None
emails_count_company = df.groupby('company').count().sort_values(by='date', ascending=False)

# See https://stackoverflow.com/questions/49574817/fill-a-new-pandas-column-with-row-numbers
emails_count_company['ranking'] = np.arange(len(emails_count_company)) + 1
emails_count_company['emails'] = emails_count_company['date']
display(emails_count_company[['ranking', 'emails']].iloc[0:20,:])
pd.options.display.max_rows = 60

In [ ]:
import plotly.express as px
import plotly.io as pio
import plotly.graph_objects as go

In [ ]:
data = go.Bar(
    x=email_count_per_ai.index, 
    y=email_count_per_ai['date'])
layout = go.Layout(
    xaxis= {'type': 'category'},
    title='{1}: Emails/AI (total {0})'.format(email_count_per_ai['date'].sum(), title))
fig = go.Figure(data=data, layout=layout)
fig.show()

fig_title = "{0} {1}".format(title, "Emails per AI")
fig.write_html( fig_title + ".html")
fig.write_image(fig_title + ".png")

In [ ]:
data = go.Bar(
    x=email_count_per_day.index, 
    y=email_count_per_day['date'])
layout = go.Layout(
    title='Emails per day (total {0})'.format(email_count_per_ai['date'].sum()),
    xaxis_tickformat = '%d.%m',
    xaxis_range = meeting_dates,
    xaxis = { 'dtick': 1000*3600*24 })
fig = go.Figure(data=data, layout=layout)
fig.show()

fig_title = "{0} {1}".format(title, "Emails per day")
fig.write_html( fig_title + ".html")
fig.write_image(fig_title + ".png")

In [ ]:
# Data source for emails/day per AI
email_count_per_day_and_ai = df.groupby(['ai', 'day']).count()

ai_plot = {}
ais = df.ai.unique()
for ai in ais:
    ai_plot[ai] = email_count_per_day_and_ai[email_count_per_day_and_ai.index.isin([ai], level=0)]['date']

layout = go.Layout(
    title='{1}: Emails/day (total {0})'.format(email_count_per_ai['date'].sum(), title),
    xaxis_tickformat = '%d.%m',
    xaxis_range = meeting_dates,
    xaxis = { 'dtick': 1000*3600*24 } )

fig = go.Figure(layout=layout)

for ai in ais:
    bar_data = ai_plot[ai]
    bar_y = bar_data
    bar_x = bar_data.index.get_level_values(1)
    bar_title = ai
    data = go.Bar(
        x=bar_x, 
        y=bar_y,
        name=bar_title)
    fig.add_trace(data)

fig.update_layout(barmode='relative')
fig.show()

fig_title = "{0} {1}".format(title, "Emails per day and AI")
fig.write_html( fig_title + ".html")
fig.write_image(fig_title + ".png")

In [ ]:
# Deactivated cell (we already have the plot per AI)
data = go.Bar(
    x=email_count_per_hour.index, 
    y=email_count_per_hour['date'])
layout = go.Layout(
    title='Emails per hour (total {0})'.format(email_count_per_ai['date'].sum()),
    xaxis_tickformat = '%d.%m  %Hh',
    xaxis_range = meeting_dates)
fig = go.Figure(data=data, layout=layout)
fig.show()

fig_title = "{0} {1}".format(title, "Emails per hour")
fig.write_html( fig_title + ".html")
fig.write_image(fig_title + ".png")

In [ ]:
# Data source for emails/h per AI
email_count_per_hour_and_ai = df.groupby(['ai', 'hour']).count()

ai_plot = {}
ais = df.ai.unique()
for ai in ais:
    ai_plot[ai] = email_count_per_hour_and_ai[email_count_per_hour_and_ai.index.isin([ai], level=0)]['date']

layout = go.Layout(
    title='{1}: Emails/hour (total {0})'.format(email_count_per_ai['date'].sum(), title),
    xaxis_tickformat = '%d.%m %Hh',
    xaxis_range = meeting_dates)

fig = go.Figure(layout=layout)

for ai in ais:
    bar_data = ai_plot[ai]
    bar_y = bar_data
    bar_x = bar_data.index.get_level_values(1)
    bar_title = ai
    data = go.Bar(
        x=bar_x, 
        y=bar_y,
        name=bar_title)
    fig.add_trace(data)

fig.update_layout(barmode='relative')
fig.show()

fig_title = "{0} {1}".format(title, "Emails per hour and AI")
fig.write_html( fig_title + ".html")
fig.write_image(fig_title + ".png")